# Experimenting with the simulated data

In [21]:
from transit_system import TransitSystem
import numpy as np
import pandas as pd
import pickle as pkl

In [22]:
# seeds = [7594897, 8032337, 3297648]
seeds = [7594897]


datas = []
analysis_interval_sec = 30
analysis_period_days = 5
hours_of_opperation_per_day = 18

for num_busses_per_route in [1, 5, 10]:
    for seed in seeds:
        transit_system = TransitSystem(
            analysis_period_sec=analysis_interval_sec,
            num_busses_per_route=num_busses_per_route,
            hours_of_opperation_per_day=hours_of_opperation_per_day,
            min_bus_capacity=100,
            max_bus_capacity=200,
            seed=seed,
        )

        for i in range(0, hours_of_opperation_per_day * 3600, analysis_interval_sec):
            for j in range(analysis_period_days):
                transit_system.step(i)

    datas.append(pd.read_csv("logs/passenger_logs.csv", low_memory=False))

with open("data.pkl", "wb") as file:
    pkl.dump(datas, file)

In [23]:
# took 70 mins to run
with open("data.pkl", "rb") as file:
    data = pkl.load(file)

In [24]:
for i in range(len(data)):
    print(f"number of passengres served by {[1, 5, 10][i]} buses per route per direction", data[i].shape[0]//5000, "thousand")
    display(data[i].groupby("num_transfers")[
    [
        "waiting_time",
        "travel_time",
        "distance_traversed",
        "total_time_taken",
        "num_stations_traversed",
        "stranding_counts"
    ]
    ].median())


number of passengres served by 1 buses per route per direction 183 thousand


,waiting_time,travel_time,distance_traversed,total_time_taken,num_stations_traversed,stranding_counts
num_transfers,,,,,,
0,390.0,480.0,40951.0,1020.0,7.0,0.0
1,360.0,1170.0,64772.0,1590.0,13.0,0.0
2,135.0,1110.0,64082.0,1410.0,13.0,0.0


number of passengres served by 5 buses per route per direction 200 thousand


,waiting_time,travel_time,distance_traversed,total_time_taken,num_stations_traversed,stranding_counts
num_transfers,,,,,,
0,390.0,480.0,41338.0,1020.0,8.0,0.0
1,360.0,1170.0,64772.0,1560.0,13.0,0.0
2,135.0,1110.0,64082.0,1410.0,13.0,0.0


number of passengres served by 10 buses per route per direction 220 thousand


,waiting_time,travel_time,distance_traversed,total_time_taken,num_stations_traversed,stranding_counts
num_transfers,,,,,,
0,390.0,480.0,41909.0,1020.0,8.0,0.0
1,330.0,1170.0,64772.0,1530.0,13.0,0.0
2,135.0,1110.0,64082.0,1410.0,13.0,0.0
